In [5]:
%pip install h2o sklearn

Defaulting to user installation because normal site-packages is not writeable
Processing ./.cache/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897/sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-1.0.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import sklearn

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_312-debug"; OpenJDK Runtime Environment (build 1.8.0_312-debug-b07); OpenJDK 64-Bit Server VM (build 25.312-b07-debug, mixed mode)
  Starting server from /home/mk7516/.local/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /state/partition1/job-19048398/tmpoue3iw5z
  JVM stdout: /state/partition1/job-19048398/tmpoue3iw5z/h2o_mk7516_started_from_python.out
  JVM stderr: /state/partition1/job-19048398/tmpoue3iw5z/h2o_mk7516_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,20 days
H2O_cluster_name:,H2O_from_python_mk7516_zve3s5
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.541 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
import pandas as pd
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(data_id=name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  y_train = y_train.to_frame(name="class")
  y_test = y_test.to_frame(name="class")
  test = pd.concat([X_test, y_test], axis=1, join='inner')
  train = pd.concat([X_train, y_train], axis=1, join='inner')
  test = h2o.H2OFrame(test)
  train = h2o.H2OFrame(train)
  return (test, train)

In [4]:
test, train = dataSetup(1596)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
train.shape

(435759, 55)

In [6]:
x = train.columns
y = "class"
x.remove(y)

In [7]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [8]:
aml = H2OAutoML(max_runtime_secs = 3600)
aml.train(x=x, y=y, training_frame=train)
# train = h2o.H2OFrame(train)

AutoML progress: |██████████████████████
15:12:17.891: GLM_1_AutoML_1_20220504_145157 [GLM def_1] failed: java.lang.ArrayIndexOutOfBoundsException: 54

█████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_1_AutoML_1_20220504_145157

No model summary for this model

ModelMetricsMultinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.16134819776683512
RMSE: 0.4016817120144196

ModelMetricsMultinomialGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.1883150193754792
RMSE: 0.4339527847306423

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.745890,0.002041,0.745322,0.743191,0.748383,0.745149,0.747404
1,auc,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
2,err,0.254110,0.002041,0.254678,0.256809,0.251617,0.254851,0.252596
3,err_count,22145.800000,123.104420,22198.000000,22282.000000,21977.000000,22209.000000,22063.000000
4,logloss,0.577859,0.002591,0.578677,0.580859,0.574617,0.579376,0.575768
5,max_per_class_error,0.762734,0.009539,0.758345,0.758304,0.779412,0.761636,0.755974
6,mean_per_class_accuracy,0.620108,0.005378,0.622726,0.615082,0.626370,0.613884,0.622479
7,mean_per_class_error,0.379892,0.005378,0.377274,0.384918,0.373630,0.386116,0.377521
8,mse,0.188317,0.000915,0.188613,0.189344,0.187123,0.188873,0.187631
9,null_deviance,210045.750000,594.555240,210708.880000,209294.530000,210561.550000,209670.020000,209993.800000


In [9]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_per_class_error,logloss,rmse,mse
StackedEnsemble_BestOfFamily_1_AutoML_1_20220504_145157,0.379936,0.577854,0.433953,0.188315
StackedEnsemble_BestOfFamily_2_AutoML_1_20220504_145157,0.381976,0.608284,0.447602,0.200347
StackedEnsemble_AllModels_1_AutoML_1_20220504_145157,0.387544,0.611949,0.448079,0.200774
GBM_1_AutoML_1_20220504_145157,0.395365,0.581051,0.434523,0.18881
XGBoost_1_AutoML_1_20220504_145157,0.412372,0.608851,0.445604,0.198563
XGBoost_2_AutoML_1_20220504_145157,0.439878,0.663945,0.47121,0.222039
DRF_1_AutoML_1_20220504_145157,0.480707,0.952469,0.480565,0.230943
GBM_4_AutoML_1_20220504_145157,0.4914,1.10899,0.657771,0.432663
GBM_3_AutoML_1_20220504_145157,0.504875,1.08154,0.647701,0.419516
GBM_2_AutoML_1_20220504_145157,0.505899,1.06132,0.639332,0.408746
